In [1]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np

class Particle:
    def __init__(self, Nseed, iq, dt, dW, x, x_prime, y, y_prime):
        self.Nseed = Nseed
        self.iq = iq
        self.dt = dt
        self.dW = dW
        self.x = x
        self.x_prime = x_prime
        self.y = y
        self.y_prime = y_prime
        self.z = 0

    def __repr__(self):
        return f"Particle(Nseed={self.Nseed}, iq={self.iq}, dt={self.dt}, dW={self.dW}, x={self.x}, x'={self.x_prime}, y={self.y}, y'={self.y_prime})"

    def propagate(self, distance):
        import math
        # print(str(self.x) + " " + str(self.y), end=" ")
        # convert angles from milliradians to radians
        angle_x = self.x_prime / 1000 
        angle_y = self.y_prime / 1000
        # print("->", end=" ")
        # calculate new positions
        self.x += distance * math.tan(angle_x)
        self.y += distance * math.tan(angle_y)
        # print(str(self.x) + " " + str(self.y))

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class Hole:
    def __init__(self, center_x, center_y, center_z, diameter):
        self.center_x = center_x
        self.center_y = center_y
        self.center_z = center_z
        self.diameter = diameter

    def __repr__(self):
        return f"Hole(center_x={self.center_x}, center_y={self.center_y}, center_z={self.center_z}, diameter={self.diameter})"
    
class Grid:
    def __init__(self, size_x, size_y, size_z, hole_diameter, separation):
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.hole_diameter = hole_diameter
        self.separation = separation
        self.holes = self.create_3d_grid()

    def create_3d_grid(self):
        holes = []
        center_x_offset = ((self.size_x - 1) * self.separation) / 2
        center_y_offset = ((self.size_y - 1) * self.separation) / 2
        center_z_offset = ((self.size_z - 1) * self.separation) / 2
        for x in range(self.size_x):
            for y in range(self.size_y):
                for z in range(self.size_z):
                    hole = Hole(center_x=x*self.separation - center_x_offset, 
                                 center_y=y*self.separation - center_y_offset, 
                                 center_z=z*self.separation - center_z_offset, 
                                 diameter=self.hole_diameter)
                    holes.append(hole)
        return holes

    def plot3D(self, ax=None):
        xs = [hole.center_x for hole in self.holes]
        ys = [hole.center_y for hole in self.holes]
        zs = [hole.center_z for hole in self.holes]
        
        if ax is None:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
        
        ax.scatter(xs, ys, zs, c='red')

    def add_to_plot3D(self, ax):
        self.plot3D(ax)

    def add_to_plot_2d(self, ax):
        # Add the grid to the 2D plot
        import matplotlib.patches as patches
        for hole in self.holes:
            ax.plot(hole.center_x, hole.center_y, 'ro')

            # Add circle representing the hole's circumference
            circle = patches.Circle((hole.center_x, hole.center_y), radius=hole.diameter/2, edgecolor='red', facecolor='none')
            ax.add_patch(circle)


    def is_point_in_a_hole(self, point):
        x1, y1, z1 = point
        hole_radius = self.hole_diameter / 2

        # Calculate the indices of the hole the point would belong to if it was in a hole
        x_index = round(x1 / self.separation)
        y_index = round(y1 / self.separation)
        z_index = round(z1 / self.separation)

        # Calculate the center of that hole
        x_center = x_index * self.separation
        y_center = y_index * self.separation
        z_center = z_index * self.separation

        # Check if the point is inside the hole
        # return ((x_center - x1)**2 + (y_center - y1)**2 + (z_center - z1)**2) <= hole_radius**2
        return (x_center - hole_radius <= x1 <= x_center + hole_radius and
                y_center - hole_radius <= y1 <= y_center + hole_radius and
                z_center - hole_radius <= z1 <= z_center + hole_radius)

data = []
with open("E://data/epsnx0.10_alfax-0.04_betax216.52_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00/coord.out", 'r') as file:
    next(file)
    for line in file:
        line = line.strip()
        if line:
            values = line.split()
            particle = Particle(
                Nseed=int(values[0]),
                iq=int(values[1]),
                dt=float(values[2]),
                dW=float(values[3]),
                x=float(values[4]),
                x_prime=float(values[5]),
                y=float(values[6]),
                y_prime=float(values[7])
            )
            data.append(particle)

size_x, size_y, size_z = 21, 21, 1
hole_diameter = 0.01
separation = 0.3

grid = Grid(size_x, size_y, size_z, hole_diameter, separation)

particles_in_holes = []
for particle in data:
    point = (particle.x, particle.y, particle.z)
    if grid.is_point_in_a_hole(point):
        particles_in_holes.append(particle)

for particle in particles_in_holes:
    particle.propagate(10)

if particles_in_holes:
    x_in_holes = [particle.x for particle in particles_in_holes]
    y_in_holes = [particle.y for particle in particles_in_holes]

bins_hist = 60

edges_hist = np.linspace(-3, 3, bins_hist + 1)

H, _, _ = np.histogram2d(x_in_holes, y_in_holes, bins=[edges_hist, edges_hist], density=True)

fig, ax = plt.subplots(figsize=(6, 6))

pcm = ax.pcolormesh(edges_hist, edges_hist, H.T, cmap='inferno')

ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.axis('square')

plt.savefig(f"images/a1.png", dpi=64, bbox_inches='tight', pad_inches=0)

plt.close(fig)

